<a href="https://colab.research.google.com/github/DivyaMeenaSundaram/Agentic-AI/blob/main/Agentic_AI_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agent with Custom Tools

In [8]:
!pip install -q langchain langchain-google-genai langchain-community google-generativeai wikipedia numexpr langchainhub

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [9]:
import random
from langchain.agents import tool


# This could be fetched dynamically from external APIs
programming_jokes = [
    "Why don't programmers like nature? It has too many bugs.",
    "Why did the C# developer fall asleep? Because he didn't like Java.",
    "['hip', 'hip'] # (hip hip array!)",
    "What's the object-oriented way to become wealthy? Inheritance.",
    "Why was the JavaScript developer sad? Because he didn't know how to 'null' his feelings.",
    "Why don't scientists trust atoms? Because they make up everything!",
    "What do you call a lazy kangaroo? Pouch potato!",
    "Why did the programmer quit his job? He didn't get arrays.",
    "How many programmers does it take to change a light bulb? None, it's a hardware problem.",
    "A SQL query goes into a bar, walks up to two tables and asks, 'Can I join you?'"
]

In [10]:
@tool
def get_programming_joke(query: str) -> str:
    """Use this tool when you are asked to tell a joke, especially about programming, software, or computers."""
    return random.choice(programming_jokes)


print("Custom 'get_programming_joke' tool created!")


Custom 'get_programming_joke' tool created!


In [15]:
from langchain.agents import load_tools
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType


from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI


GOOGLE_API_KEY = None
llm = None
try:
    GOOGLE_API_KEY = userdata.get('Divya_Key')
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0, max_tokens=500, google_api_key=GOOGLE_API_KEY)
    print("API Key loaded and LLM initialized successfully.")
except Exception as e:
    print(f"ERROR: Could not load API key or initialize LLM. {e}")
# Load existing tools
existing_tools = load_tools(["wikipedia", "llm-math"], llm=llm)


# Create the new list with ALL tools
all_tools = existing_tools + [get_programming_joke]
print(f"All loaded tools: {[t.name for t in all_tools]}")


# Initialize memory
memory_with_joke = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


API Key loaded and LLM initialized successfully.
All loaded tools: ['wikipedia', 'Calculator', 'get_programming_joke']


/tmp/ipython-input-15-3763431539.py:28: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory_with_joke = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [22]:
# Initialize the agent
agent_executor_with_joke = initialize_agent(
    all_tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory_with_joke,
    handle_parsing_errors=True # Add this to handle parsing errors gracefully
)

In [23]:
# Assuming 'agent_executor_with_joke' is created


print("\n--- Testing the Joke Tool ---")
q_joke = "Tell me a programming joke."
print(f"\nUser: {q_joke}")


response_joke = agent_executor_with_joke.invoke({"input": q_joke})
print(f"\nAgent: {response_joke['output']}")



--- Testing the Joke Tool ---

User: Tell me a programming joke.


> Entering new AgentExecutor chain...
```json
{
    "action": "get_programming_joke",
    "action_input": ""
}
```
Observation: ['hip', 'hip'] # (hip hip array!)
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The joke was: hip, hip, array!"
}
```

> Finished chain.

Agent: The joke was: hip, hip, array!


In [24]:
# Test if it still remembers/uses other tools
q_wiki = "Who is Yuval Noah Harari?"
print(f"\nUser: {q_wiki}")
response_wiki = agent_executor_with_joke.invoke({"input": q_wiki})
print(f"\nAgent: {response_wiki['output']}")



User: Who is Yuval Noah Harari?


> Entering new AgentExecutor chain...
```json
{
    "action": "wikipedia",
    "action_input": "Yuval Noah Harari"
}
```
Observation: Page: Yuval Noah Harari
Summary: Yuval Noah Harari (Hebrew: יובל נח הררי [juˈval ˈnoaχ haˈʁaʁi]; born 1976) is an Israeli medievalist, military historian, public intellectual, and popular science writer. He currently serves as professor in the Department of History at the Hebrew University of Jerusalem. His first bestselling book, Sapiens: A Brief History of Humankind (2011) is based on his lectures to an undergraduate world history class. His other works include the bestsellers Homo Deus: A Brief History of Tomorrow (2016), 21 Lessons for the 21st Century (2018), and Nexus: A Brief History of Information Networks from the Stone Age to AI (2024). His published work examines themes of free will, consciousness, intelligence, happiness, suffering and the role of storytelling in human evolution. 
In Sapiens, Harari writes a